In [2]:
import sympy as sp
import numpy as np

def from_ham_symb(N):
    """Construct an MPO for the Annealing Hamiltonian.
    
    This is an implementation using symbols to check correctness.

    Returns:
        MPO (List[array]): List of tensors representing the MPO (1 tensor for each qubit)
    """
    N_by_2 = int(N / 2)

    a_list = [np.zeros((k + 2, k + 3), dtype=object) for k in range(1, N_by_2)]

    aux = 2 if N % 2 == 0 else 3
    a_list += [np.zeros((N_by_2 + 2, N_by_2 + aux), dtype=object)]

    a_list += [np.zeros((N - k + 3, N - k + 2), dtype=object) for k in range(N_by_2 + 1, N + 1)]

    for k in range(1, N+1):
        a_list[k-1][0,0] = 1
        a_list[k-1][-1,-1] = 1
        a_list[k-1][-2,-1] = sp.Symbol(f'Z_{k}')
        a_list[k-1][0,-1] = sp.Symbol(f'(1-s)*X_{k}')*sp.Symbol(f'+s*h_{k}')*sp.Symbol(f'Z_{k}')
        
        if k < N_by_2:
            a_list[k-1][0,1] = sp.Symbol(f'Z_{k}')
            a_list[k-1][0,k+1] = sp.Symbol(f'Z_{k}')*sp.Symbol(f's*c_{k},{k+1}')

            for m in range(2, k+1):
                a_list[k-1][m-1, m] = 1
                a_list[k-1][m-1, k+1] = sp.Symbol(f's*c_{k-m+1},{k+1}')

        elif k == N_by_2:
            for n in range(2, N_by_2 + aux):
                a_list[k-1][0, n-1] = sp.Symbol(f'Z_{k}')*sp.Symbol(f's*c_{N_by_2},{N-n+2}')
                for m in range(2, k+1):
                    a_list[k-1][m-1, n-1] = sp.Symbol(f's*c_{N_by_2-m+1},{N-n+2}')

        else: # k+1 > N_by_2:
            for m in range(2, N-k+2):
                a_list[k-1][0, m-1] = sp.Symbol(f'Z_{k}')*sp.Symbol(f's*c_{k},{N-m+2}')
                a_list[k-1][m-1, m-1] = 1

    a_list[0] = a_list[0][0, :] # other rows are used to propagate information of previous tensors, does not make sense to keep them
    a_list[-1] = a_list[-1][:, -1] # other columns are used to propagate information to the next tensors, does not make sense to keep them

    tn = a_list[0]
    
    for i in range(N - 1):
        tn = np.matmul(tn, a_list[i+1])

    return tn, a_list

In [4]:
N = 3

symb_ham, a_list = from_ham_symb(N)

symb_ham

(1-s)*X_1*+s*h_1*Z_1 + (1-s)*X_2*+s*h_2*Z_2 + (1-s)*X_3*+s*h_3*Z_3 + Z_1*Z_2*s*c_1,2 + Z_3*(Z_1*s*c_1,3 + Z_2*s*c_2,3)